# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps.datasets
from pprint import pprint

# Import API key
from api_keys import g_key  




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
weather_csv_file = "./output_data/weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df = weather_df.dropna()

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bambous Virieux,-20.3428,57.7575,84.45,58,40,13.80,MU,1673524691
1,Tuatapere,-46.1333,167.6833,48.13,93,22,3.15,NZ,1673524691
2,Khatanga,71.9667,102.5000,-48.35,100,100,3.60,RU,1673524691
3,Ushuaia,-54.8000,-68.3000,51.42,87,20,6.91,AR,1673524482
4,Punta Arenas,-53.1500,-70.9167,51.91,76,0,16.11,CL,1673524692


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

In [19]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Store 'Humidity' into weights
weights = weather_df["Humidity"].astype(float)

In [20]:
# Create a humidity Heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Create a new dataframe with the cities filtered for the following conditions:
# Max temp inbetween 70 and 80F, Windspeed less than 10 mph and 0% Cloudiness.
cities_weather_fit = weather_df[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70)
                                                                      & (weather_df["Wind Speed"] < 10)
                                                                      & (weather_df["Cloudiness"] == 0) ]
cities_weather_fit

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,Puerto Escondido,15.8500,-97.0667,70.63,88,0,6.91,MX,1673524703
137,Arraial do Cabo,-22.9661,-42.0278,78.76,78,0,9.22,BR,1673524737
223,Nelson Bay,-32.7167,152.1500,73.00,82,0,8.01,AU,1673524760
317,Mangrūl Pīr,20.3167,77.3500,78.94,23,0,6.31,IN,1673524787
319,Patiya,22.2931,91.9762,75.15,38,0,5.75,BD,1673524792
418,Cayenne,4.9333,-52.3333,78.84,83,0,5.75,GF,1673524818
439,Barra de Tecoanapa,16.5167,-98.7500,71.44,81,0,3.22,MX,1673524909
479,Ranchi,23.3500,85.3333,73.51,40,0,0.00,IN,1673524928


In [22]:
# There are more than 10 cities identified and this can be further narrowed down by filtering out the cities with
# humidity greater than 85%

cities_weather_filtered = cities_weather_fit[(cities_weather_fit["Humidity"] <= 84)]
cities_weather_filtered

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
137,Arraial do Cabo,-22.9661,-42.0278,78.76,78,0,9.22,BR,1673524737
223,Nelson Bay,-32.7167,152.1500,73.00,82,0,8.01,AU,1673524760
317,Mangrūl Pīr,20.3167,77.3500,78.94,23,0,6.31,IN,1673524787
319,Patiya,22.2931,91.9762,75.15,38,0,5.75,BD,1673524792
418,Cayenne,4.9333,-52.3333,78.84,83,0,5.75,GF,1673524818
439,Barra de Tecoanapa,16.5167,-98.7500,71.44,81,0,3.22,MX,1673524909
479,Ranchi,23.3500,85.3333,73.51,40,0,0.00,IN,1673524928


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
# Store the filtered cities data into a variable named hotel_df.
hotel_df = cities_weather_filtered
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
137,Arraial do Cabo,-22.9661,-42.0278,78.76,78,0,9.22,BR,1673524737
223,Nelson Bay,-32.7167,152.1500,73.00,82,0,8.01,AU,1673524760
317,Mangrūl Pīr,20.3167,77.3500,78.94,23,0,6.31,IN,1673524787
319,Patiya,22.2931,91.9762,75.15,38,0,5.75,BD,1673524792
418,Cayenne,4.9333,-52.3333,78.84,83,0,5.75,GF,1673524818
439,Barra de Tecoanapa,16.5167,-98.7500,71.44,81,0,3.22,MX,1673524909
479,Ranchi,23.3500,85.3333,73.51,40,0,0.00,IN,1673524928


In [24]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

/var/folders/pd/tm2yj2hx5rb87k62xyb5tj6w0000gn/T/ipykernel_1105/915573950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
137,Arraial do Cabo,-22.9661,-42.0278,78.76,78,0,9.22,BR,1673524737,
223,Nelson Bay,-32.7167,152.1500,73.00,82,0,8.01,AU,1673524760,
317,Mangrūl Pīr,20.3167,77.3500,78.94,23,0,6.31,IN,1673524787,
319,Patiya,22.2931,91.9762,75.15,38,0,5.75,BD,1673524792,
418,Cayenne,4.9333,-52.3333,78.84,83,0,5.75,GF,1673524818,
439,Barra de Tecoanapa,16.5167,-98.7500,71.44,81,0,3.22,MX,1673524909,
479,Ranchi,23.3500,85.3333,73.51,40,0,0.00,IN,1673524928,


In [25]:
# find a hotel within 5000m radius for each city's coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up the required parameters
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
#   get lat and lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    
#   add location to params dict
    params["location"] = f"{lat},{lng}"
    
#   assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#   extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel in {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
       
    except (KeyError, IndexError):
        print(f"No hotel at {row['City']}")
        hotel_df.loc[index, "Hotel Name"] = "NA"
        
    print("------------")

Closest hotel in Arraial do Cabo is Orlanova Boutique Hotel.
------------
Closest hotel in Nelson Bay is Ramada Resort by Wyndham Shoal Bay.
------------
No hotel at Mangrūl Pīr
------------
No hotel at Patiya
------------
Closest hotel in Cayenne is Grand Hotel Montabo.
------------
Closest hotel in Barra de Tecoanapa is Hotel Garapacho.
------------
Closest hotel in Ranchi is Capitol Hill.
------------


In [26]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
137,Arraial do Cabo,-22.9661,-42.0278,78.76,78,0,9.22,BR,1673524737,Orlanova Boutique Hotel
223,Nelson Bay,-32.7167,152.1500,73.00,82,0,8.01,AU,1673524760,Ramada Resort by Wyndham Shoal Bay
317,Mangrūl Pīr,20.3167,77.3500,78.94,23,0,6.31,IN,1673524787,NA
319,Patiya,22.2931,91.9762,75.15,38,0,5.75,BD,1673524792,NA
418,Cayenne,4.9333,-52.3333,78.84,83,0,5.75,GF,1673524818,Grand Hotel Montabo
439,Barra de Tecoanapa,16.5167,-98.7500,71.44,81,0,3.22,MX,1673524909,Hotel Garapacho
479,Ranchi,23.3500,85.3333,73.51,40,0,0.00,IN,1673524928,Capitol Hill


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Orlanova Boutique Hotel</dd>\n<dt>City</dt><dd>Arraial do Cabo</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Ramada Resort by Wyndham Shoal Bay</dd>\n<dt>City</dt><dd>Nelson Bay</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>NA</dd>\n<dt>City</dt><dd>Mangrūl Pīr</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>NA</dd>\n<dt>City</dt><dd>Patiya</dd>\n<dt>Country</dt><dd>BD</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Grand Hotel Montabo</dd>\n<dt>City</dt><dd>Cayenne</dd>\n<dt>Country</dt><dd>GF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Garapacho</dd>\n<dt>City</dt><dd>Barra de Tecoanapa</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Capitol Hill</dd>\n<dt>City</dt><dd>Ranchi</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n']

In [29]:
# Add marker layer on top of the humidity heat map created earlier.
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))